**ML Exercise 2: Applying existing deep learning models**

In this exercise, we will apply an existing deep learning model EQTransformer (Mousavi et al. 2020) for earthquake detection. We will also explore some great resources (e.g. the USGS earthquake catalog) that can be useful for all members of our seismological community, not just our ML enthusiasts.

Note: We apply EQTransformer with its Seisbench default parameters here. A detailed description of its available parameters are provided in the original study (Mousavi et al. 2020) and an evaluation study by Pita-Sllim et al. (2023). See references at the end. 

**<center> PART I: Earthquake Detection <center>** 

1. Run the following cell blocks to load the necessary Python packages and the original EQTransformer model.

In [ ]:
# import required packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seisbench.models as sbm
from obspy.clients.fdsn import Client
client = Client("IRIS")
from obspy import UTCDateTime

In [ ]:
# load the EQTransformer model
EQT = sbm.EQTransformer.from_pretrained("original")
print(EQT.weights_docstring)

2. The original EQTransformer (EQT) model developed by Mousavi et al. (2020) was trained on the STanford EArthquake Dataset (STEAD, Mousavi et al. 2019). The python package *Seisbench* can be used to train the EQT model on different datasets. Some variations of EQT are already available for use. Explore the [Seisbench docs](https://seisbench.readthedocs.io/en/stable/) to find the function that returns a list of pretrained models of EQT. Benchmark datasets available through Seisbench are described [here](https://seisbench.readthedocs.io/en/stable/pages/benchmark_datasets.html).  

In [ ]:
# print list of pretrained models of EQT
# INSERT function

3. Run the cell below to retrieve and plot 6 hours of data from station PR.GBPR and applying EQTransformer. Use [this EarthScope tool](https://ds.iris.edu/gmap/) to locate this station. A metadata summary of station PR.GBPR is available here: http://ds.iris.edu/mda/PR/GBPR/. If you are unfamiliar with seismic station paramaters (e.g. location code, channel code), please ask the instructor or TA for an overview. Detailed information on naming conventions are given [here](https://www.geonet.org.nz/data/supplementary/channels) and [here](http://ds.iris.edu/ds/nodes/dmc/tools/data_channels/#???). 

In [ ]:
# retrieve and plot data (this may take a minute or so)
start_t = UTCDateTime("2024-04-28T12:00:00")
dur = 60*60*6
stream = client.get_waveforms(network="PR", station="GBPR", location="00", 
                              channel="HH?", starttime=start_t, endtime=start_t+dur)
stream.plot();

4. Run the cell below to apply EQTransformer to our 3-component stream and print the output of classify().

In [ ]:
# apply EQT (this may take a minute or so)
output = EQT.classify(stream)
print(output)

5. In *Seisbench*, we can use the function classify() or annotate() to apply EQTransformer. Explore the docs to determine the difference between the outputs of classify() and annotate(). Write your answer in the markdown cell below. 

**Insert your answer here**

6. As you saw in the previous code cell, the output of classify() includes a separate list of phase picks and a separate list of detections. Phase picks are not associated with a particular detection. In the cell below, we define a function that creates a [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) of EQT detections, associated phases, and other descriptors. The function also estimates source distances and origin times using the associated phase picks. Run the cells below to create the dataframe for our 6-hour data of PR.GBPR. Read through the function create_df() and create a concise description of the function.

    Note that some detections may not have associated phase picks. It is also possible that a detection may include multiple P or S picks, which the function does not consider (see Bonus 2).

In [ ]:
def create_df(output):
    '''
    Insert concise description that addresses the parameters and output of this function. 
    '''
    num_events = len(output.detections)

    # store arrays/lists of event ids, starttimes, endtimes, peak detection confidence
    event_ids = np.arange(0, num_events)
    event_starttimes = [ detection.start_time for detection in output.detections]
    event_endtimes = [ detection.end_time for detection in output.detections]
    event_maxconf = [ detection.peak_value for detection in output.detections]

    # find P/S times within the detection duration and add it to the appropriate event 
    event_Ptimes = [np.nan] * num_events
    event_Stimes = [np.nan] * num_events
    for pick in output.picks:
        for event_id in event_ids:
            if pick.peak_time >= event_starttimes[event_id] and pick.peak_time < event_endtimes[event_id]:
                if pick.phase == 'P':
                    event_Ptimes[event_id] = pick.peak_time
                    break
                if pick.phase == 'S':
                    event_Stimes[event_id] = pick.peak_time
                    break
    
    # estimate a preliminary origin time and source distance using simple relations/velocities
    event_estotime = [np.nan] * num_events
    event_estdist = [np.nan] * num_events
    for event_id, P_time in enumerate(event_Ptimes):
        S_time = event_Stimes[event_id]
        if pd.isnull(P_time) or pd.isnull(S_time):
            continue
        dist = (S_time - P_time) * 8.4                                 #distances in km
        event_estdist[event_id] = dist
        event_estotime[event_id] = P_time - (dist/6)

    # create dataframe 
    event_dict = {'event_ID':event_ids, 'start_time':event_starttimes, 
                'end_time':event_endtimes, 'detect_maxconf':event_maxconf,
                'P': event_Ptimes, 'S': event_Stimes, 
                'estimated_sourcedist_km': event_estdist,
                'estimated_origintime': event_estotime}
    df = pd.DataFrame(event_dict)
    return df

In [ ]:
# create dataframe of EQT classifications
df = create_df(output)
df

7. Using the [USGS earthquake catalog](https://earthquake.usgs.gov/earthquakes/search/), generate a CSV (comma-separarated values) file of catalogued earthquakes near station PR.GBPR in our 6-hour time period. Search for all earthquakes with magnitudes greater than or equal to 0 and use a custom region that is roughly a 300 km radius of PR.GBPR.

8. Using your preferred editor (e.g. Microsoft Excel, Google Sheets, TextEdit), open the CSV file and create a new column called "detected_by_EQT" which will be filled with Booleans (TRUE or FALSE). Flag 'True' if the USGS-listed event has an origin time within 60 seconds of any estimated EQT origin time in our dataframe. For each flagged event, estimate (by eye) the distance between the USGS-listed event epicenter and station PR.GBPR using this [USGS map](https://earthquake.usgs.gov/earthquakes/map/?extent=14.38148,-71.10352&extent=21.93285,-62.57813&range=search&sort=oldest&timeZone=utc&search=%7B%22name%22:%22Search%20Results%22,%22params%22:%7B%22starttime%22:%222024-04-28%2012:00:00%22,%22endtime%22:%222024-04-28%2018:00:00%22,%22maxlatitude%22:23.003,%22minlatitude%22:14.651,%22maxlongitude%22:-57.942,%22minlongitude%22:-74.905,%22minmagnitude%22:0,%22orderby%22:%22time%22%7D%7D). Check if this distance is comparable to our estimated source distances (column 'estimated_sourcedist_km' in the dataframe) of EQT detections. If so, it is most likely that we have detected this listed earthquake using EQT.

    For EQT detections without estimated origin times and distances (when they do not have associated P and S picks), you may compare the origin time with the detection start time.


9. With just one station and a few lines of code, you have detected a majority of the nearby earthquakes listed in USGS! Keep in mind that geolgical surveys and data centers usually use a multi-station detection approach so this is a great achievement. However, we do see that some catalogued quakes were not detected by EQT. Let us visualize them in GBPR data. Pick a USGS-catalogued earthquake that was flagged 'False'. Use the cell below to plot the event in its unfiltered and high-pass filtered versions. Try different time durations and filter frequencies to make the best plots. 

    Note that we make copies of stream objects. If we filter our stream, the original raw data will be no longer accessible afterwards. We apply processing routines on copies of the stream object so that the original, raw 3-component data will be always available to us. 

In [ ]:
time = UTCDateTime('#INSERT TIME')
stream_copy_unfilt = stream.copy()
stream_copy_unfilt.plot(starttime= time - 0, 
            endtime=time + 30);

# some events are only clear if we filter our data. plot a high-passed filtered version
print('High-pass filtered')
stream_copy_filt = stream.copy()
stream_copy_filt.filter('highpass', freq=2, zerophase=True)
stream_copy_filt.plot(starttime= time - 0, 
            endtime=time + 30);

10. Plot some other USGS-catalogued earthquakes. Compare the flagged 'True' and 'False' earthquakes (earthquakes detected by EQT vs earthquakes missed by EQT). How do they differ in their waveforms and source parameters (magnitude, distance to GBPR)? If you were to publish a study about the performance of EQT on GBPR data, what type of statistics and figures would you provide?

**Insert your answer here**

11. We also see that there are some EQT detections that are not listed in the USGS catalog. Could they be uncatalogued earthquakes? Plot some of these events and document your thoughts below. 

In [ ]:
# space for code as needed

**Insert your answer here**

**<center> PART II: Phase picking <center>** 

In this short section, we will explore the performance of EQT's phase picking tasks. Manual phase picking can be a tedious and challenging task for analysts. Automating this process can speed up this process and allows us to analyze larger datasets efficiently. Compared to traditional phase picking methods (e.g. STA/LTA ratio method), EQT also promises a better performance for noisy and incomplete data. However, it is important for us to validate these picks. Especially in noisy settings, automated pickers can mix up P and S phases and detect false positive picks. 

1. Let us first look into the closest earthquake to station PR.GBPR: M2.9 earthquake about 11 km S of Tallaboa, Puerto Rico. Find the USGS webpage for this earthquake. In the code cell below, fill out the missing information (earthquake origin time and station sampling rate) to plot the earthquake using the plotting package matplotlib. Be sure to include the most precise origin time available. 

In [ ]:
stream_copy1 = stream.copy()
samp_rate = #INSERT SAMPLING RATE                               # samping rate (in Hz) of PR.GBPR
ref_time = UTCDateTime('#INSERT ORGIN TIME')
ref_time_sample = int( (ref_time - stream[0].stats.starttime) * samp_rate)

duration_s = 15
times_array_s = np.arange(0,duration_s,1/samp_rate)
duration_npts = int(duration_s * samp_rate)

fig, ax = plt.subplots(3,1)
for i, chan in enumerate(['HHZ', 'HH1', 'HH2']):
    tr_data = stream_copy1.select(channel=chan)[0].data[ref_time_sample:ref_time_sample+duration_npts]
    ax[i].plot(times_array_s,tr_data, label=chan, color='black', linewidth=1)
    ax[i].set_xlim(times_array_s[0], times_array_s[-1])
    ax[i].legend(loc='upper left')

ax[2].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()


2. Using your knowledge of body waves, pick your own P and S arrivals and plot them (using vertical lines) in the figure above. Feel free to adjust the time limits so that you can make your best estimates. Explain your justification in a 1-2 sentences. 

**Insert your answer here**

3. Plot the phase picks predicted by EQT. Add appropriate labels and helpful colors/linestyles to create a clear figure. How does your picks compare to those of EQT?

4. Let us now compare these phase picks with those listed in USGS. In the 'Origin' section of this earthquake's USGS webpage, you will see a 'Phases' tab. Find the P and S arrivals for our station GBPR and plot them in your figure. How do the three picks compare?

5. Repeat steps 1-4 for the M3.6 earthquake that occurred 78 km N of Cruz Bay, U.S. Virgin Islands. Picking your phases may be a bit more challenging here. Describe your thoughts and comparisons in the cell below. 

    Hint: You may need to change the plotting duration and/or filter your data. 

In [ ]:
# space for code as needed

**Insert your answer here**

**<center> Bonus exercises <center>** 

1. Recall that one of the listed pretrained models is "original_nonconvervative". Explore the Github page of EQTransformer to determine the difference between the conservative and nonconservative models. Apply the nonconservative EQT to our 6-hour stream data with the recommended thresholds. Can this model detect the "False" flagged events (earthquakes missed by EQT's conservative model)? Were the results what you expected?

2. The simple function *create_df()* does not account for the case when multiple P or S arrivals may be picked within one event duration. When there are multiple P or S picks in one event, which one is stored to our dataframe (if we keep our function as is)? How can we modify this function to account for multiple phase picks? Modify the function with your best idea. 

3. Seisbench is an excellent package because you can easily apply multiple models and compare their performance with eachother. Apply the model PhaseNet (Zhu and Beroza, 2019) to our data and compare its phase picks with that of EQT. Seisbench has easy-to-follow examples [here](https://seisbench.readthedocs.io/en/stable/pages/examples.html) not only on applying existing models but also on creating a dataset, an event catalog, and other applications. 

Congratultions on making it this far! If you revisit this exercise after the workshop and have any questions or concerns, please do not hesitate to reach out to Ann (annthomas2025@u.northwestern.edu). 

References

Beyreuther, M., Barsch, R., Krischer, L. et al. ObsPy: A Python Toolbox for Seismology. Seismological Research Letters 81(3), 530–533 (2010). https://doi.org/10.1785/gssrl.81.3.530

Mousavi, S.M., Sheng, Y., Zhu, W. & Beroza, G. C. "STanford EArthquake Dataset (STEAD): A Global Data Set of Seismic Signals for AI. IEEE Access 7, 179464-179476 (2019), https://doi.org/10.1109/ACCESS.2019.2947848

Mousavi, S.M., Ellsworth, W.L., Zhu, W. et al. Earthquake transformer—an attentive deep-learning model for simultaneous earthquake detection and phase picking. Nat Commun 11, 3952 (2020). https://doi.org/10.1038/s41467-020-17591-w

Pita‐Sllim, O., Chamberlain, C.J, Townend, J., & Warren‐Smith, E. Parametric Testing of EQTransformer’s Performance against a High‐Quality, Manually Picked Catalog for Reliable and Accurate Seismic Phase Picking. The Seismic Record 3(4), 332–341 (2023). https://doi.org/10.1785/0320230024

Woollam, J., Münchmeyer, J., Tilmann, F. et al. SeisBench—A Toolbox for Machine Learning in Seismology. Seismological Research Letters 93(3), 1695–1709 (2022). https://doi.org/10.1785/0220210324

Zhu, W., & Beroza, G.C. PhaseNet: a deep-neural-network-based seismic arrival-time picking method. Geophysical Journal International 216(1), 261–273 (2019). https://doi.org/10.1093/gji/ggy423
